In [1]:
import numpy as np
from scipy.spatial import distance_matrix
from gurobipy import *
#from scipy.spatial import ConvexHull
#from shapely.geometry import Polygon, Point
from numpy import random
import time
import pandas as pd
from itertools import zip_longest   # zip 함수 길이 다를 경우
from haversine import haversine

# 행정동 입력
* 행정동: 후보 행정동 입력
* 행정동리스트: 후보 행정동 인근 행정동 입력(후보 행정동 포함)

In [34]:
행정동 = '장위2동'
행정동리스트 = ['장위2동', '월곡2동', '석관동', '월계1동', '장위3동', '장위1동']

In [35]:
# 행정동 = '역촌동'
# 행정동리스트 = ['역촌동', '신사1동', '응암1동', '대조동', '구산동', '응암2동']

In [36]:
# 행정동 = '홍제3동'
# 행정동리스트 = ['홍제3동', '홍제2동', '홍제1동', '홍은2동', '홍은1동', '부암동']

In [37]:
# 행정동 = '반포본동'
# 행정동리스트 = ['반포본동', '방배본동', '사당2동', '반포2동', '반포4동']

In [38]:
입지후보지 = pd.read_csv('수정_최종후보입지.csv', encoding='EUC-KR')
입지후보지 = 입지후보지[입지후보지['행정동'] == 행정동]

버스 = pd.read_csv('서울시_버스정류장_행정동추가.csv')
버스 = 버스[버스['행정동'].isin(행정동리스트)]
버스_points = np.array([list(i) for i in zip(버스['X좌표'], 버스['Y좌표'])])

지하철 = pd.read_csv('서울시_지하철역_행정동추가.csv', encoding='EUC-KR', index_col=0)
지하철 = 지하철[지하철['행정동'].isin(행정동리스트)]
지하철_points = np.array([list(i) for i in zip(지하철['경도'], 지하철['위도'])])

주차장 = pd.read_csv('서울시_주차장_행정동추가.csv', encoding='EUC-KR', index_col=0).drop('Unnamed: 0.1', axis=1)
주차장 = 주차장[주차장['행정동'].isin(행정동리스트)]
주차장_points = np.array([list(i) for i in zip(주차장['경도'], 주차장['위도'])])

주택 = pd.read_csv('서울시주택_행정동추가.csv', encoding='EUC-KR', index_col=0).drop('Unnamed: 0.1', axis=1)
주택 = 주택[주택['행정동'].isin(행정동리스트)]
주택_points = np.array([list(i) for i in zip(주택['위도'], 주택['경도'])])   # 위도 경도 바뀜

In [39]:
# 버스 = pd.read_csv('서울시_버스정류장_행정동추가.csv')
# 버스['행정동'].nunique()

In [40]:
# print(입지후보지.shape)
# print(버스.shape)
# print(지하철.shape)
# print(주차장.shape)
# print(주택.shape)

In [41]:
X = list(버스['X좌표']) + list(지하철['경도']) + list(주차장['경도']) + list(주택['위도'])     # 주택 위도 경도 이름 바뀜
Y = list(버스['Y좌표']) + list(지하철['위도']) + list(주차장['위도']) + list(주택['경도'])

In [42]:
points = np.array([list(i) for i in zip(X, Y)])
print(points.shape)
points[:2]

(145, 2)


array([[127.06723071,  37.61506526],
       [127.06700402,  37.61519435]])

# 가중치 수정
* 가중치: (그 요소 제외 합) / (전체 합)

In [43]:
전체w = points.shape[0]
버스w = 버스.shape[0]
지하철w = 지하철.shape[0]
주차장w = 주차장.shape[0]
주택w = 주택.shape[0]

In [44]:
m1 = (전체w-버스w)/전체w
m2 = (전체w-지하철w)/전체w
m3 = (전체w-주차장w)/전체w
m4 = (전체w-주택w)/전체w

# 기본 MCLP

In [45]:
# def mclp(points,K,radius):
#     """
#     Solve maximum covering location problem
#     Input:
#         points: 버스정류장, 지하철역 위치 좌표 등 (기타 인근에 있으면 좋은 시설 좌표)
#         K: 배치할 노인놀이터의 수
#         radius: 반경 (노인들이 이동하기 적합한 거리)
#         M: generate_candidate_sites 함수에서 생성할 random 좌표 수 (임의의 노인놀이터 수)
#         the ConvexHull wrapped by the polygon
#     Return:
#         opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
#         f: the optimal value of the objective function
#     """
#     print('  Number of points %g' % points.shape[0])
#     print('  K %g' % K)
#     print('  Radius %g' % radius)

#     start = time.time()
#     sites = np.array([list(i) for i in zip(입지후보지['x좌표'], 입지후보지['y좌표'])])
#     J = sites.shape[0]                                         # 임의의 노인놀이터 좌표 개수
#     I = points.shape[0]                                        # 버스정류장, 지하철역 위치 좌표 개수
#     D = distance_matrix(points,sites)                          # 노인놀이터와 버스/지하철역 좌표 간 거리 계산
#     mask1 = D<=radius
#     D[mask1]=1                                                 # 버스/지하철역 좌표별로 노인놀이터의 반경 내 속하면 1, 아니면 0
#     D[~mask1]=0
#     # Build model
#     m = Model()
#     # Add variables
#     x = {}
#     y = {}
#     for i in range(I):                                        # 버스/지하철역 좌표 개수만큼 반복
#         y[i] = m.addVar(vtype=GRB.BINARY, name="y%d" % i)     # 모델에 결정 변수 추가 (변수 유형은 BINARY. 위치별 인수, 변수 이름 y+숫자)
#     for j in range(J):
#         x[j] = m.addVar(vtype=GRB.BINARY, name="x%d" % j)     # 노인놀이터 좌표 개수만큼 반복하여 변수 추가

#     m.update()
#     # Add constraints
#     m.addConstr(quicksum(x[j] for j in range(J)) == K)

#     for i in range(I):
#         m.addConstr(quicksum(x[j] for j in np.where(D[i]==1)[0]) >= y[i])

#     m.setObjective(quicksum(y[i] for i in range(I)),GRB.MAXIMIZE)
#     m.setParam('OutputFlag', 0)
#     m.optimize()
#     end = time.time()
#     print('----- Output -----')
#     print('  Running time : %s seconds' % float(end-start))
#     print('  Optimal coverage points: %g' % m.objVal)
    
#     print(quicksum(y[i] for i in range(I)))
#     print(I) # 129개 -> 수정 후 106개..
    
#     solution = []
#     if m.status == GRB.Status.OPTIMAL:
#         for v in m.getVars():
#             # print v.varName,v.x
#             if v.x==1 and v.varName[0]=="x":
#                 solution.append(int(v.varName[1:]))
#     opt_sites = sites[solution]
#     return opt_sites,m.objVal

In [46]:
# opts_sites, mobjVal = mclp(points, 3, 0.0027)
# opts_sites

# MCLP + 목적함수 수정

In [47]:
# def mclp2(버스_points, 지하철_points, 주차장_points, 주택_points, points, K, radius):
#     """
#     Solve maximum covering location problem
#     Input:
#         points: 버스정류장, 지하철역 위치 좌표 등 (기타 인근에 있으면 좋은 시설 좌표)
#         K: 배치할 노인놀이터의 수
#         radius: 반경 (노인들이 이동하기 적합한 거리)
#         M: generate_candidate_sites 함수에서 생성할 random 좌표 수 (임의의 노인놀이터 수)
#         the ConvexHull wrapped by the polygon
#     Return:
#         opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
#         f: the optimal value of the objective function
#     """
#     print('  Number of points %g' % points.shape[0])
#     print('  K %g' % K)
#     print('  Radius %g' % radius)

#     start = time.time()
#     sites = np.array([list(i) for i in zip(입지후보지['x좌표'], 입지후보지['y좌표'])])
#     J = sites.shape[0]                                         # 후보지 수
    
#     # 수요지점 수
#     A = 버스_points.shape[0]
#     B = 지하철_points.shape[0]
#     C = 주차장_points.shape[0]
#     D = 주택_points.shape[0]
    
#     # 후보지와 수요지점 간 거리 계산
#     D1 = distance_matrix(버스_points, sites)
#     D2 = distance_matrix(지하철_points, sites)
#     D3 = distance_matrix(주차장_points, sites)
#     D4 = distance_matrix(주택_points, sites)
    
#     for i in [D1, D2, D3, D4]:
#         mask1 = i<=radius
#         i[mask1]=1                                                 # 반경 내 속하면 1, 아니면 0
#         i[~mask1]=0

#     m = Model()
#     x1, x2, x3, x4 = {}, {}, {}, {}
#     y = {}
    
#     # 수요지점 변수 추가
#     for i in range(A):                                       
#         x1[i] = m.addVar(vtype=GRB.BINARY, name="x1%d" % i)
#     for i in range(B):                                       
#         x2[i] = m.addVar(vtype=GRB.BINARY, name="x2%d" % i)
#     for i in range(C):                                       
#         x3[i] = m.addVar(vtype=GRB.BINARY, name="x3%d" % i)
#     for i in range(D):                                       
#         x4[i] = m.addVar(vtype=GRB.BINARY, name="x4%d" % i)
    
#     for j in range(J):
#         y[j] = m.addVar(vtype=GRB.BINARY, name="y%d" % j)     # 후보지 변수 추가

#     m.update()
#     m.addConstr(quicksum(y[j] for j in range(J)) == K)        # 후보지 제약 조건 (3개)

#     # 수요지점 제약 조건
#     for i in range(A): 
#         m.addConstr(quicksum(y[j] for j in np.where(D1[i]==1)[0]) >= x1[i])
#     for i in range(B): 
#         m.addConstr(quicksum(y[j] for j in np.where(D2[i]==1)[0]) >= x2[i])
#     for i in range(C): 
#         m.addConstr(quicksum(y[j] for j in np.where(D3[i]==1)[0]) >= x3[i])
#     for i in range(D): 
#         m.addConstr(quicksum(y[j] for j in np.where(D4[i]==1)[0]) >= x4[i])

#     # 목적함수 수정
# #     m.setObjective(quicksum(1*x1[a] + 1*x2[b] + 1*x3[c] + 1*x4[d] \
# #                             for a,b,c,d in zip_longest(range(A),range(B),range(C),range(D), fillvalue=0)),\
# #                    GRB.MAXIMIZE)   # 제약조건 포함, 모든 수요지점 더하기

#     res=[]
#     for a,b,c,d in zip_longest(range(A),range(B),range(C),range(D), fillvalue=0):
#         w1=1; w2=1; w3=1; w4=1
#         if a==b==c==d==0:
#             w1=1;w2=1;w3=1;w4=1
#         else:
#             if b==0:
#                 w2=0
#             if c==0:
#                 w3=0
#             if d==0:
#                 w4=0
#         res.append(w1*x1[a] + w2*x2[b] + w3*x3[c] + w4*x4[d])

#     m.setObjective(quicksum(i for i in res),GRB.MAXIMIZE)
#     m.setParam('OutputFlag', 0)
#     m.optimize()
#     end = time.time()
#     print('----- Output -----')
#     print('  Running time : %s seconds' % float(end-start))
#     print('  Optimal coverage points: %g' % m.objVal)
    
#     #print(quicksum(i for i in res))
#     #print(res)
#     #print(len(res))
    
#     solution = []
#     if m.status == GRB.Status.OPTIMAL:
#         for v in m.getVars():
#             # print v.varName,v.x
#             if v.x==1 and v.varName[0]=="y":
#                 solution.append(int(v.varName[1:]))
#     opt_sites = sites[solution]
#     return opt_sites, m.objVal

In [48]:
# opts_sites, mobjVal = mclp2(버스_points, 지하철_points, 주차장_points, 주택_points, points, 3, 0.0027)
# opts_sites

# MCLP + Haversine

In [49]:
# haversine -> meter 단위로 수정
def mclp3(버스_points, 지하철_points, 주차장_points, 주택_points, points, K, radius):
    
    """
    Solve maximum covering location problem
    Input:
        points: 버스정류장, 지하철역 위치 좌표 등 (기타 인근에 있으면 좋은 시설 좌표)
        K: 배치할 노인놀이터의 수
        radius: 반경 (노인들이 이동하기 적합한 거리)
        M: generate_candidate_sites 함수에서 생성할 random 좌표 수 (임의의 노인놀이터 수)
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)

    start = time.time()
    sites = np.array([list(i) for i in zip(입지후보지['x좌표'], 입지후보지['y좌표'])])
    J = sites.shape[0]                                         # 후보지 수
    
    # 수요지점 수
    A = 버스_points.shape[0]
    B = 지하철_points.shape[0]
    C = 주차장_points.shape[0]
    D = 주택_points.shape[0]
    
    # 후보지와 수요지점 간 거리 계산
    D1 = []
    for i in 버스_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D1.append(site)
    D1 = np.array(D1)
    
    D2 = []
    for i in 지하철_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D2.append(site)
    D2 = np.array(D2)    
    
    D3 = []
    for i in 주차장_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D3.append(site)
    D3 = np.array(D3)
    
    D4 = []
    for i in 주택_points:
        site = []
        for j in sites:
            site.append(haversine(i, j)*1000)
        D4.append(site)
    D4 = np.array(D4)
    
    for i in [D1, D2, D3, D4]:
        mask1 = i<=radius
        i[mask1]=1                                                 # 반경 내 속하면 1, 아니면 0
        i[~mask1]=0

    m = Model()
    x1, x2, x3, x4 = {}, {}, {}, {}
    y = {}
    
    # 수요지점 변수 추가
    for i in range(A):                                       
        x1[i] = m.addVar(vtype=GRB.BINARY, name="x1%d" % i)
    for i in range(B):                                       
        x2[i] = m.addVar(vtype=GRB.BINARY, name="x2%d" % i)
    for i in range(C):                                       
        x3[i] = m.addVar(vtype=GRB.BINARY, name="x3%d" % i)
    for i in range(D):                                       
        x4[i] = m.addVar(vtype=GRB.BINARY, name="x4%d" % i)
    
    for j in range(J):
        y[j] = m.addVar(vtype=GRB.BINARY, name="y%d" % j)     # 후보지 변수 추가

    m.update()
    m.addConstr(quicksum(y[j] for j in range(J)) == K)        # 후보지 제약 조건

    # 수요지점 제약 조건
    for i in range(A): 
        m.addConstr(quicksum(y[j] for j in np.where(D1[i]==1)[0]) >= x1[i])
    for i in range(B): 
        m.addConstr(quicksum(y[j] for j in np.where(D2[i]==1)[0]) >= x2[i])
    for i in range(C): 
        m.addConstr(quicksum(y[j] for j in np.where(D3[i]==1)[0]) >= x3[i])
    for i in range(D): 
        m.addConstr(quicksum(y[j] for j in np.where(D4[i]==1)[0]) >= x4[i])
    
    # 목적함수 수정
    res=[]
    for a,b,c,d in zip_longest(range(A),range(B),range(C),range(D), fillvalue=0):
        w1=m1;w2=m2;w3=m3;w4=m4
        if a==b==c==d==0:
            w1=m1;w2=m2;w3=m3;w4=m4
        else:
            if b==0:
                w2=0
            if c==0:
                w3=0
            if d==0:
                w4=0
        res.append(w1*x1[a] + w2*x2[b] + w3*x3[c] + w4*x4[d])

    m.setObjective(quicksum(i for i in res),GRB.MAXIMIZE)
    
    m.setParam('OutputFlag', 0)
    m.optimize()
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objVal)
    
    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            # print v.varName,v.x
            if v.x==1 and v.varName[0]=="y":
                solution.append(int(v.varName[1:]))
    opt_sites = sites[solution]
    return opt_sites,m.objVal

In [50]:
opts_sites, mobjVal = mclp3(버스_points, 지하철_points, 주차장_points, 주택_points, points, 3, 300)
opts_sites

  Number of points 145
  K 3
  Radius 300
----- Output -----
  Running time : 0.01595282554626465 seconds
  Optimal coverage points: 8.41379


array([[127.054738 ,  37.612296 ],
       [127.051625 ,  37.614919 ],
       [127.052577 ,  37.6093012]])

# 시각화
* 단, meter 단위 아님
* 0.0027 degrees ~= 300m

In [51]:
def plot_input(points):
    '''
    Plot the result
    Input:
        points: input points, Numpy array in shape of [N,2]
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        radius: the radius of circle
    '''
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(8,8))
    plt.scatter(points[:,0],points[:,1],c='C0')
    ax = plt.gca()
    ax.axis('equal')
    ax.tick_params(axis='both',left=False, top=False, right=False,
                       bottom=False, labelleft=False, labeltop=False,
                       labelright=False, labelbottom=False)

In [30]:
def plot_result(points,opt_sites,radius):
    '''
    Plot the result
    Input:
        points: input points, Numpy array in shape of [N,2]
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        radius: the radius of circle
    '''
    from matplotlib import pyplot as plt
    fig = plt.figure(figsize=(8,8))
    plt.scatter(points[:,0],points[:,1],c='C0')
    ax = plt.gca()
    plt.scatter(opt_sites[:,0],opt_sites[:,1],c='C1',marker='+')
    for site in opt_sites:
        circle = plt.Circle(site, radius, color='C1',fill=False,lw=2)
        ax.add_artist(circle)
    ax.axis('equal')
    ax.tick_params(axis='both',left=False, top=False, right=False,
                       bottom=False, labelleft=False, labeltop=False,
                       labelright=False, labelbottom=False)

In [31]:
plot_result(points, opts_sites, 0.0027)

# csv 파일 생성
* 행정동은 바꿔주세요

In [85]:
후보지 = pd.DataFrame(opts_sites, columns=['경도','위도'])
후보지.to_csv('장위2동_후보지.csv', index=False, encoding='cp949')

In [84]:
수요지점 = pd.DataFrame(points, columns=['경도','위도'])
수요지점.to_csv('장위2동_수요지점.csv', index=False, encoding='cp949')

In [86]:
버스_수요지점 = pd.DataFrame(버스_points, columns=['경도','위도'])
버스_수요지점.to_csv('장위2동_버스수요지점.csv', index=False, encoding='cp949')

In [87]:
지하철_수요지점 = pd.DataFrame(지하철_points, columns=['경도','위도'])
지하철_수요지점.to_csv('장위2동_지하철수요지점.csv', index=False, encoding='cp949')

In [88]:
주차장_수요지점 = pd.DataFrame(주차장_points, columns=['경도','위도'])
주차장_수요지점.to_csv('장위2동_주차장수요지점.csv', index=False, encoding='cp949')

In [89]:
주택_수요지점 = pd.DataFrame(주택_points, columns=['경도','위도'])
주택_수요지점.to_csv('길동_주택수요지점.csv', index=False, encoding='cp949')